# POS Audit: Category-Level Constraint Analysis

This notebook demonstrates that syntactic structure shifts probability mass toward grammatically-appropriate POS categories.

**Expected Pattern:**
- After "the" (determiner) → High % NOUN/ADJ in Sentence & Jabberwocky
- After "the" in Scrambled → Lower % (structure disrupted)

## Step 1: Install Dependencies

In [ ]:
!pip install -q transformers torch

## Step 2: Upload Stimuli File

**IMPORTANT:** Click the folder icon on the left sidebar, then drag and drop your `stimuli_with_scrambled.json` file into the Files area.

Or run this cell to upload:

In [ ]:
from google.colab import files
uploaded = files.upload()  # Click "Choose Files" and select stimuli_with_scrambled.json

## Step 3: Define POS Tagger

In [ ]:
def simple_pos_tag(word):
    """Simple rule-based POS tagger"""
    word_lower = word.lower().strip()
    
    if word_lower in ['the', 'a', 'an']:
        return 'DET'
    if word_lower in ['in', 'on', 'at', 'to', 'for', 'with', 'from', 'by']:
        return 'PREP'
    if word_lower in ['i', 'you', 'he', 'she', 'it', 'we', 'they']:
        return 'PRON'
    if word_lower in ['is', 'are', 'was', 'were', 'has', 'have', 'had']:
        return 'AUX'
    if word_lower.endswith('ly'):
        return 'ADV'
    if word_lower.endswith('ing') or word_lower.endswith('ed'):
        return 'VERB'
    
    # Default: assume noun
    return 'NOUN'

## Step 4: Load Model and Stimuli

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Loading GPT-2...")
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.eval()
print("Model loaded successfully!\n")

print("Loading stimuli...")
with open('stimuli_with_scrambled.json') as f:
    stimuli = json.load(f)
print(f"Loaded {len(stimuli)} stimulus sets.\n")

## Step 5: Run POS Audit

In [ ]:
def analyze_cue(model, tokenizer, text, cue_word, expected_pos, k=50):
    """Analyze predictions after a diagnostic cue"""
    words = text.split()
    
    # Find cue word
    cue_position = None
    for i, word in enumerate(words[:-1]):
        if word.lower().strip('.,!?;:') == cue_word.lower():
            cue_position = i
            break
    
    if cue_position is None:
        return None
    
    # Get context
    context = ' '.join(words[:cue_position+1])
    inputs = tokenizer(context, return_tensors='pt')
    
    # Get top-k predictions
    with torch.no_grad():
        outputs = model(**inputs)
        next_token_logits = outputs.logits[0, -1, :]
        probs = torch.softmax(next_token_logits, dim=-1)
        top_k_probs, top_k_ids = torch.topk(probs, k)
    
    # Decode and tag
    candidates = []
    for prob, token_id in zip(top_k_probs, top_k_ids):
        token_str = tokenizer.decode([token_id], skip_special_tokens=True).strip()
        if token_str:
            candidates.append({
                'token': token_str,
                'prob': prob.item(),
                'pos': simple_pos_tag(token_str)
            })
    
    # Compute % in expected categories
    total_prob = sum(c['prob'] for c in candidates)
    expected_prob = sum(c['prob'] for c in candidates if c['pos'] in expected_pos)
    expected_pct = (expected_prob / total_prob * 100) if total_prob > 0 else 0
    
    return {
        'cue_position': cue_position,
        'cue_word': words[cue_position],
        'expected_categories': list(expected_pos),
        'expected_pct': expected_pct,
        'top_5': [(c['token'], c['pos'], f"{c['prob']*100:.1f}%") for c in candidates[:5]]
    }

print("="*80)
print("POS AUDIT RESULTS")
print("="*80)
print()

# Analyze first 5 stimuli for "the" → NOUN/ADJ pattern
for stim_idx in range(min(5, len(stimuli))):
    stim = stimuli[stim_idx]
    
    print(f"\nSTIMULUS {stim_idx + 1}:")
    print("-" * 80)
    
    for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
        text = stim[condition]
        print(f"\n{condition.upper()}:")
        print(f"  Text: {text[:60]}...")
        
        result = analyze_cue(model, tokenizer, text, 'the', {'NOUN', 'ADJ'})
        
        if result:
            print(f"  Expected categories (NOUN/ADJ): {result['expected_pct']:.1f}%")
            print(f"  Top-5 predictions:")
            for token, pos, prob in result['top_5']:
                print(f"    - {token:15s} [{pos:6s}] {prob}")
        else:
            print("  (No 'the' found in this text)")

print("\n" + "="*80)
print("INTERPRETATION:")
print()
print("If syntax constrains predictions:")
print("  - Sentence & Jabberwocky should show HIGH % for expected categories")
print("  - Scrambled should show LOWER % (structure disrupted)")
print()
print("This demonstrates category-level constraint beyond specific lexical items.")
print("="*80)

## Step 6: Download Results (Optional)

Run this cell to save and download the full analysis:

In [ ]:
# Save detailed results to JSON
results = []

for stim_idx, stim in enumerate(stimuli):
    stim_results = {'stimulus_idx': stim_idx}
    
    for condition in ['sentence', 'jabberwocky_matched', 'scrambled_jabberwocky']:
        text = stim[condition]
        result = analyze_cue(model, tokenizer, text, 'the', {'NOUN', 'ADJ'})
        
        if result:
            stim_results[condition] = {
                'expected_pct': result['expected_pct'],
                'top_5': result['top_5']
            }
    
    results.append(stim_results)

# Save to file
with open('pos_audit_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to pos_audit_results.json")

# Download file
from google.colab import files
files.download('pos_audit_results.json')
print("Download started!")

## Summary Statistics

In [ ]:
import numpy as np

# Compute averages across all stimuli
sentence_pcts = []
jab_pcts = []
scrambled_pcts = []

for stim in stimuli:
    for condition, pcts_list in [('sentence', sentence_pcts), 
                                   ('jabberwocky_matched', jab_pcts),
                                   ('scrambled_jabberwocky', scrambled_pcts)]:
        text = stim[condition]
        result = analyze_cue(model, tokenizer, text, 'the', {'NOUN', 'ADJ'})
        if result:
            pcts_list.append(result['expected_pct'])

print("\n" + "="*80)
print("SUMMARY STATISTICS: % Probability on NOUN/ADJ after 'the'")
print("="*80)
print()
print(f"Sentence:              {np.mean(sentence_pcts):.1f}% ± {np.std(sentence_pcts):.1f}%  (n={len(sentence_pcts)})")
print(f"Jabberwocky (matched): {np.mean(jab_pcts):.1f}% ± {np.std(jab_pcts):.1f}%  (n={len(jab_pcts)})")
print(f"Scrambled:             {np.mean(scrambled_pcts):.1f}% ± {np.std(scrambled_pcts):.1f}%  (n={len(scrambled_pcts)})")
print()
print(f"Δ (Jabberwocky - Scrambled): {np.mean(jab_pcts) - np.mean(scrambled_pcts):+.1f}%")
print("="*80)